# SRT 현황 분석

2024.01.11 ~ 2024.01.19

---

## 1. 프로젝트 개요

### 1-1. 프로젝트 요약
- 제목: 제목
- 설명: 설명
- 사용한 데이터셋
    - [(주)에스알_SRT 승차권 발권](https://www.data.go.kr/data/15108354/openapi.do): 국민철도SR 승차권 발매현황 데이터로, "발매채널별 승차권 발매 현황", "SR 운영역 승차권 발매현황", "연도별 SR 승차권 발매현황", "특실/일반실 승차인원" 항목이 있습니다.
- 진행 절차 및 내용
    - 내용1
    - 내용2
- 사용한 스킬셋
    - pandas
    - matplotlib, seaborn
    - ...

### 1-2. 프로젝트 진행 배경

내용

---

## 2. 데이터 로딩 및 EDA

### 2-1. API 테스트

In [41]:
import requests
import json
import pickle

import pandas as pd

#### 2-1-1. channel_sales

발매채널별 승차권 발매현황

In [47]:
with open('service_key.pkl', 'rb') as file:
    service_key = pickle.load(file)

In [48]:
url = 'https://apis.data.go.kr/B553912/tk_sales/v1/channel_sales'

params = {
    'serviceKey': service_key,
    'page': 1,
    'perPage': 100,
    'returnType': 'json',
    'cond[RUN_YMD::GTE]': '20231001', 
    'cond[RUN_YMD::LT]': '20231231'
}

response = requests.get(url, params=params)
json.loads(response.content)

{'currentCount': 100,
 'data': [{'PRT_CMPTN_QNTY': 0,
   'RFND_QNTY': 1,
   'ROUTE_NM': '경부선',
   'RUN_YMD': '20231001',
   'SELLNG_QNTY': 1,
   'SELLNG_WICKET_KND_CD': '0085',
   'SELLNG_WICKET_KND_NM': 'INTERNET',
   'SELLNG_YMD': '20230831'},
  {'PRT_CMPTN_QNTY': 0,
   'RFND_QNTY': 4,
   'ROUTE_NM': '경부선',
   'RUN_YMD': '20231001',
   'SELLNG_QNTY': 4,
   'SELLNG_WICKET_KND_CD': '0077',
   'SELLNG_WICKET_KND_NM': '스마트폰',
   'SELLNG_YMD': '20230905'},
  {'PRT_CMPTN_QNTY': 0,
   'RFND_QNTY': 6,
   'ROUTE_NM': '호남선',
   'RUN_YMD': '20231001',
   'SELLNG_QNTY': 6,
   'SELLNG_WICKET_KND_CD': '0077',
   'SELLNG_WICKET_KND_NM': '스마트폰',
   'SELLNG_YMD': '20230905'},
  {'PRT_CMPTN_QNTY': 0,
   'RFND_QNTY': 1,
   'ROUTE_NM': '경부선',
   'RUN_YMD': '20231001',
   'SELLNG_QNTY': 1,
   'SELLNG_WICKET_KND_CD': '0085',
   'SELLNG_WICKET_KND_NM': 'INTERNET',
   'SELLNG_YMD': '20230906'},
  {'PRT_CMPTN_QNTY': 32,
   'RFND_QNTY': 42,
   'ROUTE_NM': '경부선',
   'RUN_YMD': '20231001',
   'SELLNG_QNTY': 74,

In [43]:
pd.DataFrame(json.loads(response.content)['data'])

,PRT_CMPTN_QNTY,RFND_QNTY,ROUTE_NM,RUN_YMD,SELLNG_QNTY,SELLNG_WICKET_KND_CD,SELLNG_WICKET_KND_NM,SELLNG_YMD
0,0,1,경부선,20231001,1,0085,INTERNET,20230831
1,0,4,경부선,20231001,4,0077,스마트폰,20230905
2,0,6,호남선,20231001,6,0077,스마트폰,20230905
3,0,1,경부선,20231001,1,0085,INTERNET,20230906
4,32,42,경부선,20231001,74,0011,역,20230907
...,...,...,...,...,...,...,...,...
95,4,1,경부선,20231001,5,0027,철도공사 자회사 위탁운영역,20230916
96,0,1,경부선,20231001,1,0037,Tablet(자동발매기),20230916
97,390,709,경부선,20231001,1107,0077,스마트폰,20230916
98,6,17,경부선,20231001,23,0085,INTERNET,20230916


각 데이터가 무엇을 의미하는지를 알아야... 하는데...
- ```PRT_CMPTN_QNTY```: ?
- ```RFND_QNTY```: ...? 환불 이런 것 같기도
- ```ROUTE_NM```: 노선 이름
- ```RUN_YMD```: 열차 운영 날짜를 YYYYMMDD 형태로 표현
- ```SELLING_QNTY```: 판매 개수(한 번에 몇 장 구매했는지)
- ```SELLING_WICKET_KND_CD```: 판매 경로 코드
- ```SELLING_WICKET_KIN_NM```: 판매 경로
- ```SELLING_YMD```: 판매 날짜를 YYYYMMDD 형태로 표현

#### 2-1-2. srt_station_sales

SR 운영역 승차권 발매현황

In [44]:
url = 'https://apis.data.go.kr/B553912/tk_sales/v1/srt_station_sales'

params = {
    'serviceKey': service_key,
    'page': 1,
    'perPage': 100,
    'returnType': 'json',
    'cond[RUN_YMD::GTE]': '20231001', 
    'cond[RUN_YMD::LT]': '20231231'
}

response = requests.get(url, params=params)
json.loads(response.content)

{'currentCount': 100,
 'data': [{'PRT_CMPTN_QNTY': 20,
   'RFND_QNTY': 19,
   'ROUTE_NM': '경부선',
   'RUN_YMD': '20231001',
   'SELLNG_QNTY': 39,
   'SELLNG_STN_CD': '0551',
   'SELLNG_STN_NM': '수서',
   'SELLNG_YMD': '20230907'},
  {'PRT_CMPTN_QNTY': 6,
   'RFND_QNTY': 13,
   'ROUTE_NM': '경부선',
   'RUN_YMD': '20231001',
   'SELLNG_QNTY': 19,
   'SELLNG_STN_CD': '0552',
   'SELLNG_STN_NM': '동탄',
   'SELLNG_YMD': '20230907'},
  {'PRT_CMPTN_QNTY': 6,
   'RFND_QNTY': 6,
   'ROUTE_NM': '경부선',
   'RUN_YMD': '20231001',
   'SELLNG_QNTY': 12,
   'SELLNG_STN_CD': '0553',
   'SELLNG_STN_NM': '지제',
   'SELLNG_YMD': '20230907'},
  {'PRT_CMPTN_QNTY': 10,
   'RFND_QNTY': 10,
   'ROUTE_NM': '호남선',
   'RUN_YMD': '20231001',
   'SELLNG_QNTY': 20,
   'SELLNG_STN_CD': '0551',
   'SELLNG_STN_NM': '수서',
   'SELLNG_YMD': '20230907'},
  {'PRT_CMPTN_QNTY': 23,
   'RFND_QNTY': 12,
   'ROUTE_NM': '호남선',
   'RUN_YMD': '20231001',
   'SELLNG_QNTY': 35,
   'SELLNG_STN_CD': '0552',
   'SELLNG_STN_NM': '동탄',
   'SELL

In [45]:
pd.DataFrame(json.loads(response.content)['data'])

,PRT_CMPTN_QNTY,RFND_QNTY,ROUTE_NM,RUN_YMD,SELLNG_QNTY,SELLNG_STN_CD,SELLNG_STN_NM,SELLNG_YMD
0,20,19,경부선,20231001,39,0551,수서,20230907
1,6,13,경부선,20231001,19,0552,동탄,20230907
2,6,6,경부선,20231001,12,0553,지제,20230907
3,10,10,호남선,20231001,20,0551,수서,20230907
4,23,12,호남선,20231001,35,0552,동탄,20230907
...,...,...,...,...,...,...,...,...
95,4,0,경부선,20231001,4,0553,지제,20230926
96,2,8,호남선,20231001,10,0551,수서,20230926
97,3,0,호남선,20231001,3,0552,동탄,20230926
98,1,1,호남선,20231001,2,0553,지제,20230926


이번 데이터의 경우
- ```PRT_CMPTN_QNTY```: ?
- ```RFND_QNTY```: ?
- ```ROUTE_NM```: 노선 이름
- ```RUN_YMD```: 열차 운영 날짜를 YYYYMMDD 형태로 표현
- ```SELLING_QNTY```: 판매 개수(한 번에 몇 장 구매했는지)
- ```SELLING_STN_CD```: 판매 역 코드
- ```SELLING_STN_NM```: 판매 역
- ```SELLING_YMD```: 판매 날짜를 YYYYMMDD 형태로 표현

유저가 현장 발매 창구를 통해 구매한 것만을 의미하는지, 혹은 인터넷으로 구매한 유저라도 해당 역에서 탑승했다면 해당 역으로 집계되는 것인지 확인이 필요할 것 같다.

#### 2-1-3. yearly_sales

연도별 SR 승차권 발매현황

In [52]:
url = 'https://apis.data.go.kr/B553912/tk_sales/v1/yearly_sales'

params = {
    'serviceKey': service_key,
    'page': 1,
    'perPage': 100,
    'returnType': 'json',
    'cond[RUN_YR::GTE]': '2018', 
    'cond[RUN_YR::LTE]': '2023'
}

response = requests.get(url, params=params)
json.loads(response.content)

{'currentCount': 4,
 'data': [{'PRT_CMPTN_QNTY': 7243153,
   'RFND_QNTY': 4992679,
   'ROUTE_NM': '경부선',
   'RUN_YR': '2022',
   'SELLNG_QNTY': 12330420},
  {'PRT_CMPTN_QNTY': 2803787,
   'RFND_QNTY': 2002003,
   'ROUTE_NM': '호남선',
   'RUN_YR': '2022',
   'SELLNG_QNTY': 4854346},
  {'PRT_CMPTN_QNTY': 17978897,
   'RFND_QNTY': 13335563,
   'ROUTE_NM': '경부선',
   'RUN_YR': '2023',
   'SELLNG_QNTY': 31550478},
  {'PRT_CMPTN_QNTY': 7160244,
   'RFND_QNTY': 5544814,
   'ROUTE_NM': '호남선',
   'RUN_YR': '2023',
   'SELLNG_QNTY': 12823553}],
 'matchCount': 4,
 'page': 1,
 'perPage': 100,
 'totalCount': 4}

In [53]:
pd.DataFrame(json.loads(response.content)['data'])

,PRT_CMPTN_QNTY,RFND_QNTY,ROUTE_NM,RUN_YR,SELLNG_QNTY
0,7243153,4992679,경부선,2022,12330420
1,2803787,2002003,호남선,2022,4854346
2,17978897,13335563,경부선,2023,31550478
3,7160244,5544814,호남선,2023,12823553


이번 데이터의 경우
- ```PRT_CMPTN_QNTY```: 무슨 전체 양 이런 것 같다
- ```RFND_QNTY```: 그럼 이건 환불된 양일까
- ```ROUTE_NM```: 노선 이름
- ```RUN_YR```: 열차 운영 연도를 YYYY 형태로 표현
- ```SELLING_QNTY```: 판매 개수(총 몇 장 구매했는지)

시작연도를 2018년으로 넣었는데 2022년 이후 데이터밖에 안 준다.

#### 2-1-4. room_class_passengers

특실/일반실 승차인원

In [57]:
url = 'https://apis.data.go.kr/B553912/tk_sales/v1/room_class_passengers'

params = {
    'serviceKey': service_key,
    'page': 1,
    'perPage': 100,
    'returnType': 'json',
    'cond[RUN_YM::GTE]': '202201', 
    'cond[RUN_YM::LT]': '202312'
}

response = requests.get(url, params=params)
json.loads(response.content)

{'currentCount': 64,
 'data': [{'PSRM_GRD_CD': '1',
   'PSRM_GRD_NM': '일반실',
   'ROUTE_NM': '경부선',
   'RUN_YM': '202208',
   'TKCAR_NMPR_CNT': 1145743},
  {'PSRM_GRD_CD': '2',
   'PSRM_GRD_NM': '특실',
   'ROUTE_NM': '경부선',
   'RUN_YM': '202208',
   'TKCAR_NMPR_CNT': 78352},
  {'PSRM_GRD_CD': '1',
   'PSRM_GRD_NM': '일반실',
   'ROUTE_NM': '호남선',
   'RUN_YM': '202208',
   'TKCAR_NMPR_CNT': 433382},
  {'PSRM_GRD_CD': '2',
   'PSRM_GRD_NM': '특실',
   'ROUTE_NM': '호남선',
   'RUN_YM': '202208',
   'TKCAR_NMPR_CNT': 32316},
  {'PSRM_GRD_CD': '1',
   'PSRM_GRD_NM': '일반실',
   'ROUTE_NM': '경부선',
   'RUN_YM': '202209',
   'TKCAR_NMPR_CNT': 1328839},
  {'PSRM_GRD_CD': '2',
   'PSRM_GRD_NM': '특실',
   'ROUTE_NM': '경부선',
   'RUN_YM': '202209',
   'TKCAR_NMPR_CNT': 103303},
  {'PSRM_GRD_CD': '1',
   'PSRM_GRD_NM': '일반실',
   'ROUTE_NM': '호남선',
   'RUN_YM': '202209',
   'TKCAR_NMPR_CNT': 509760},
  {'PSRM_GRD_CD': '2',
   'PSRM_GRD_NM': '특실',
   'ROUTE_NM': '호남선',
   'RUN_YM': '202209',
   'TKCAR_NMPR_CNT': 

In [58]:
pd.DataFrame(json.loads(response.content)['data'])

,PSRM_GRD_CD,PSRM_GRD_NM,ROUTE_NM,RUN_YM,TKCAR_NMPR_CNT
0,1,일반실,경부선,202208,1145743
1,2,특실,경부선,202208,78352
2,1,일반실,호남선,202208,433382
3,2,특실,호남선,202208,32316
4,1,일반실,경부선,202209,1328839
...,...,...,...,...,...
59,2,특실,호남선,202310,48108
60,1,일반실,경부선,202311,1347644
61,2,특실,경부선,202311,102267
62,1,일반실,호남선,202311,591675


이번 데이터의 경우
- ```PSRM_GRD_CD```: 특실, 일반실 구분 코드 (일반실 = 1, 특실 = 2)
- ```PSRM_GRD_NM```: 특실, 일반실 구분
- ```ROUTE_NM```: 노선 이름
- ```RUN_YM```: 열차 운영 날짜를 YYYYMM 형태로 표현
- ```SELLING_QNTY```: 판매 개수(총 몇 장 구매했는지)

월별로 집계된 데이터고, 202201부터 달라고 했는데 202208부터 준다.

### 2-2. 데이터 로딩

(다음 주 진행 예정)
4개 데이터 로딩. 아마 channel_sales가 가장 큰 데이터셋일 듯

In [39]:
url = 'https://apis.data.go.kr/B553912/tk_sales/v1/channel_sales'
results = []
i = 0
per_page = 20000

while True:
    params = {
        'serviceKey': service_key,
        'page': i,
        'perPage': per_page,
        'returnType': 'json',
        'cond[RUN_YMD::GTE]': '20231001',
        'cond[RUN_YMD::LT]': '20231231'
    }

    response = requests.get(url, params=params)
    result = json.loads(response.content)['data']
    
    results += result
    print(f"데이터 불러오는 중 ... 현재 개수: {len(results)}")
    
    if len(result) < per_page:
        break
    
    i += 1

데이터 불러오는 중 ... 현재 개수: 20000
데이터 불러오는 중 ... 현재 개수: 40000
데이터 불러오는 중 ... 현재 개수: 46126


In [35]:
results[-1]

{'PRT_CMPTN_QNTY': 76,
 'RFND_QNTY': 24,
 'ROUTE_NM': '호남선',
 'RUN_YMD': '20231230',
 'SELLNG_QNTY': 107,
 'SELLNG_WICKET_KND_CD': '0085',
 'SELLNG_WICKET_KND_NM': 'INTERNET',
 'SELLNG_YMD': '20231230'}

In [24]:
len(results)

50000

### 2-3. 데이터 분포 확인 및 전처리

In [ ]:
# 분포 확인, 시각화, 전처리

In [ ]:
# pickle로 저장

---

## 3. 데이터 분석

### 3-1. 문제 정의

내용

### 3-2. 데이터 분석

In [ ]:
# 가설 검정, 머신러닝, ...

### 3-3. 결과 해석

내용

---

## 4. 후기

내용